# Data preparation

## Data do datasetu s 5minutovým intervalem

### Data z FVE

In [74]:
import pandas as pd
from datetime import timedelta


df_generation = pd.read_csv("data\solax\datacsvnaexport_final.csv", sep = ";")
df_generation['update time'] = df_generation['update time'].str.rstrip('.')
df_generation['timestamp'] = pd.to_datetime(df_generation['update time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

### Data z meteostanic 

In [75]:
# load
primary_df = pd.read_csv('data\wunderground\IVELKO9.csv')
secondary_df = pd.read_csv('data\wunderground\IPODBR33.csv')

# pridan nazev stanice
primary_df = primary_df.assign(dataset='IVELKO9')
secondary_df = secondary_df.assign(dataset='IPODBR33')

# vytvoren timestamp sloupec
primary_df['timestamp'] = primary_df['Date'] + ' ' + primary_df['Time']
secondary_df['timestamp'] = secondary_df['Date'] + ' ' + secondary_df['Time']
# konvert
primary_df['timestamp'] = pd.to_datetime(primary_df['timestamp'], format='%Y/%m/%d %I:%M %p', errors='coerce')
secondary_df['timestamp'] = pd.to_datetime(secondary_df['timestamp'], format='%Y/%m/%d %I:%M %p', errors='coerce')
# drop NAT
primary_df = primary_df.dropna(subset=['timestamp'])
secondary_df = secondary_df.dropna(subset=['timestamp'])
# zaokrouhleno
primary_df["rounded_timestamp"] = primary_df["timestamp"].dt.round("5min")
secondary_df["rounded_timestamp"] = secondary_df["timestamp"].dt.round("5min")

# reseni missing values vypadku stanic a concat
primary_mask = primary_df['Temperature_C'].isna()
missing_timestamps = primary_df.loc[primary_mask, 'rounded_timestamp']
secondary_subset = secondary_df[secondary_df['rounded_timestamp'].isin(missing_timestamps)]
merged_df = pd.concat([primary_df, secondary_subset], ignore_index=True)

# seřadit podle rounded_timestamp pro časovou posloupnost
merged_df = merged_df.sort_values('rounded_timestamp')

#drop zbytečné sloupce
merged_df_dropped = merged_df.drop(columns=['Date','Time',])

# rounded_timestamp na index0 pro přehlednost
cols = list(merged_df_dropped.columns)
cols.insert(0, cols.pop(cols.index('rounded_timestamp')))
merged_df_dropped = merged_df.loc[:, cols]


df_weather = merged_df_dropped.dropna(subset=['Temperature_C'])

### Spojení dataset
Napojovat se bude na timestamp, takže je provedeno zaokrouhlení na 5min

In [76]:
df_reference = pd.read_csv('data_final/reference_table.csv')
#df_consumption = pd.read_csv('data/cez_data_elektromer/pnd_spotreba3.csv',sep = ";", encoding='ISO-8859-1')


df_generation.drop(['EPS active power R(W)','EPS active power S(W)','EPS active power T(W)','EPS apparent power R(VA)','EPS apparent power S(VA)'
                    ,'EPS apparent power T(VA)'], axis=1, inplace=True)


# fce na zaokrouhlení
def custom_rounding(timestamp, interval):
    delta = timedelta(minutes=interval)
    remainder = timestamp.minute % interval
    if remainder < interval / 2:
        rounded = timestamp - timedelta(minutes=remainder)
    elif remainder == interval / 2:
        rounded = timestamp - timedelta(minutes=remainder)
    else:
        rounded = timestamp + (delta - timedelta(minutes=remainder))
    return rounded.replace(second=0, microsecond=0)

# timestamp na datetime format
df_reference['timestamp'] = pd.to_datetime(df_reference['datetime'])
df_generation['timestamp'] = pd.to_datetime(df_generation['timestamp'])
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])

#call rounding fce
df_generation['timestamp_rounded'] = df_generation['timestamp'].apply(lambda x: custom_rounding(x, 5))
df_weather['timestamp_rounded'] = df_weather['timestamp'].apply(lambda x: custom_rounding(x, 5))

df_reference.set_index('timestamp', inplace=True)
df_generation.set_index('timestamp_rounded', inplace=True)
df_weather.set_index('timestamp_rounded', inplace=True)

df_merged_with_reference = pd.merge(df_reference, df_generation, how='left', left_index=True, right_index=True)
df_final_merged = pd.merge(df_merged_with_reference, df_weather, how='left', left_index=True, right_index=True)

df_final_merged.index.name = 'timestamp'

#drop nepotrebnych
df_final_merged.drop(['update time','timestamp_x','rounded_timestamp','timestamp_y','datetime'], axis=1, inplace=True)

df_final_merged.to_csv("merged_5min_.csv")

C:\Users\lukas\AppData\Local\Temp\ipykernel_19528\589364323.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])
C:\Users\lukas\AppData\Local\Temp\ipykernel_19528\589364323.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather['timestamp_rounded'] = df_weather['timestamp'].apply(lambda x: custom_rounding(x, 5))


## Vytvoření datasetu s intervaly 1d

# dict na resampling metodu
resample_dict = {
    'daily yield(kWh)': 'max',
    'consume energy(kWh)': 'last',
    'feed-in energy(kWh)': 'last',
    'PV1 input power(W)': 'max',
    'PV2 input power(W)': 'max',
    'feed-in power(W)': 'min',
    'feed-in power(W)': 'max',
}

# resample
df_filtered = df_final_merged.between_time('00:05', '23:55')
daily_df = df_filtered.resample('D').agg(resample_dict)

df_filtered['Temperature_C'] = pd.to_numeric(df_filtered['Temperature_C'], errors='coerce')

daily_df['peak_production_time'] = df_filtered['output power(W)'].resample('D').apply(lambda x: x.idxmax().time())
daily_df['peak_PV1_production_time'] = df_filtered['PV1 input power(W)'].resample('D').apply(lambda x: x.idxmax().time())
daily_df['peak_PV2_production_time'] = df_filtered['PV2 input power(W)'].resample('D').apply(lambda x: x.idxmax().time())
daily_df['peak_feed_inpower_time'] = df_filtered['feed-in power(W)'].resample('D').apply(lambda x: x.idxmax().time())
daily_df['low_feed_inpower_time'] = df_filtered['feed-in power(W)'].resample('D').apply(lambda x: x.idxmin().time())
daily_df['average_temperature'] = df_filtered['Temperature_C'].resample('D').mean()
daily_df['total_precipitation'] = df_filtered['Precip_Accum_mm'].resample('D').last()

# kalkulace feature
daily_df['daily_consumption'] = daily_df['consume energy(kWh)'] - df_final_merged['consume energy(kWh)'].resample('D').first()
daily_df['daily_feed_in_energy'] = daily_df['feed-in energy(kWh)'] - df_final_merged['feed-in energy(kWh)'].resample('D').first()

# drop zbytecnychy
daily_df.drop(columns=['consume energy(kWh)', 'feed-in energy(kWh)'], inplace=True)

daily_df.to_csv("daily.csv")



## Kategorizace atributu pro CleverMiner

In [95]:
import pandas as pd
df = pd.read_csv('merged_5min.csv')

print(df.columns)

Index(['timestamp', 'PV1 voltage (V)', 'PV1 current (A)', 'PV1 input power(W)',
       'PV2 voltage (V)', 'PV2 current (A)', 'PV2 input power(W)',
       'AC current R(A)', 'AC voltage R(V)', 'AC current S(A)',
       'AC voltage S(V)', 'AC current T(A)', 'AC voltage T(V)',
       'output power(W)', 'feed-in power(W)', 'daily yield(kWh)',
       'total yield(kWh)', 'feed-in energy(kWh)', 'consume energy(kWh)',
       'Inverter Status', 'Battery operating status', 'Temperature_C',
       'Dew_Point_C', 'Humidity_%', 'Wind', 'Speed_kmh', 'Gust_kmh',
       'Pressure_hPa', 'Precip_Rate_mm', 'Precip_Accum_mm', 'UV', 'Solar_w/m2',
       'dataset'],
      dtype='object')


In [96]:
columns_to_convert = ['PV1 voltage (V)', 'PV1 current (A)', 'PV1 input power(W)',
       'PV2 voltage (V)', 'PV2 current (A)', 'PV2 input power(W)',
       'AC current R(A)', 'AC voltage R(V)', 'AC current S(A)',
       'AC voltage S(V)', 'AC current T(A)', 'AC voltage T(V)',
       'output power(W)', 'feed-in power(W)', 'daily yield(kWh)',
       'total yield(kWh)', 'feed-in energy(kWh)', 'consume energy(kWh)',
       'Temperature_C', 'Dew_Point_C', 'Humidity_%', 'Speed_kmh', 'Gust_kmh',
       'Pressure_hPa', 'Precip_Rate_mm', 'Precip_Accum_mm', 'UV', 'Solar_w/m2']


def replace_zeros(value):
    if isinstance(value, str):
        if value.strip() == '0':
            return '0.0'
        # Replace commas with dots for decimal conversion
        return value.replace(',', '.')
    return value

# Apply the function to the entire DataFrame
df = df.applymap(replace_zeros)

for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce')

df.to_csv('df.csv')



In [94]:
from pandas_profiling import ProfileReport
# Generate the profile report
profile = ProfileReport(df, title='Pandas Profiling Report', minimal=True)

# Save the report to a file
profile.to_file("your_data_profile.html")

# You can also display the report within a Jupyter Notebook
profile.to_notebook_iframe()


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

# Define bin edges and labels
bins = [0, 15, 35, 55]  # Adjust these values for your cutoffs
labels = ['Low', 'Medium', 'High']  # Names for the categories

# Use the cut function to categorize the values
df['categories'] = pd.cut(df['numerical_values'], bins=bins, labels=labels)

print(df)
